In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score, zero_one_loss
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime

pd.set_option('display.max_columns', None)

In [2]:
art = pd.read_csv('clients_art_wo_names.csv')
# Я предварительно поменял в исходном файле десятичный разделитель с запятой на точку
print(art[:5])
print('Shape')
print(art.shape)
print('Unique')
for column in art.columns:
    print("{:8s}\t".format(column), art[column].unique().shape[0])
# Считаем, что артикул art_id - просто случайное число, а из кодов групп в дереве товаров
# можно получить какую-то метрику. Я предполагаю, что вода, хлеб и крупа не окажутся 
# одновременно в одной группе товаров третьего уровня, а условные вода "Шишкин лес" и "Святой
# источник окажутся в одной группе lvl2_id, а lvl3_id у них будет отличаться. 


     art_id  weight  lvl0_id  lvl1_id  lvl2_id  lvl3_id
0    192584    0.21    16752     1677     7263    15232
1    310536    1.00    16352       34    16258      574
2  15321862    0.25    16722      655    16094    16092
3    114853    0.60     2135     1953     1962     7961
4  16522966    0.50      679      912      970    14862
Shape
(12382, 6)
Unique
art_id  	 12382
weight  	 486
lvl0_id 	 35
lvl1_id 	 185
lvl2_id 	 693
lvl3_id 	 1511


In [3]:
train = pd.read_csv('clients_train.csv')
test = pd.read_csv('clients_test_cut.csv')
print('Train')
print(train[:3])
print('Test')
print(test[:3])
print('----------------------------------------')
print('Unique values')
print('Column\t\tTrain\t\tTest')
print('Shape\t\t', train.shape, '\t', test.shape)
for column in test.columns:
    print("{:8s}\t".format(column), train[column].unique().shape[0], '\t\t', test[column].unique().shape[0])
print("\n{:8s}\t".format('client_id'), train['client_id'].unique().shape[0])
print('----------------------------------------')
# Посмотрим, есть ли пересечения в данных
print('Test in train')
for column in test.columns:
    counts = test[column].isin(train[column]).value_counts()
    print("{:8s}\t\t".format(column), counts[counts.index == True].values)

Train
       day_id    art_id  whs_id       txn_id  sale_qnty  reg_opsum  \
0  2018-02-04  16550635   26517  14914458657      1.000     85.900   
1  2018-02-03    243028   26517  14905137106      1.000    149.900   
2  2018-02-03    327275   26517  14895612196      0.173     60.533   

   fact_opsum  mission_id  client_id  
0      85.900          -1  116871314  
1     149.900          -1  126493766  
2      60.533          -1  105675535  
Test
       day_id    art_id  whs_id       txn_id  sale_qnty  reg_opsum  \
0  2018-04-01    275845   26517  15578368583        1.0       24.9   
1  2018-04-01  16751680   26517  15578368583        1.0       29.9   
2  2018-04-01  15333128   26517  15578368583        1.0       19.9   

   fact_opsum  mission_id  
0        24.9         220  
1        29.9         220  
2        19.9         220  
----------------------------------------
Unique values
Column		Train		Test
Shape		 (8093, 9) 	 (3581, 8)
day_id  	 59 		 30
art_id  	 2151 		 1304
whs_id  	 1 

In [4]:
train = pd.read_csv('clients_train.csv')
test = pd.read_csv('clients_test_cut.csv')

def gen_features(X):
    X = X.copy()
    X = X.join(art.set_index('art_id'), on = 'art_id')
    # Артикул можно бы и удалить, но градиентный бустинг достаточно устойчив
    # С лишними признаками разберемся потом
    # X.drop(['art_id'], inplace = True, axis = 1)    
    # Одно число в столбце. Для обучения модели оно не поможет
    X.drop('whs_id', inplace = True, axis = 1)
    # Так как и количество, и вес могут быть дробными, введем еще одну переменную
    # Почему-то в некоторых местах вес или количество равны 0
    X['sale_qnty'] = X['sale_qnty'].map(lambda x: 1 if x == 0 else x)
    X['weight'] = X['weight'].map(lambda x: 1 if x == 0 else x)
    X['qty'] = X['sale_qnty'].multiply(X['weight'])
    # X.drop(['sale_qnty', 'weight'], inplace = True, axis = 1)
    X['prod_disc'] = (X['reg_opsum'] - X['fact_opsum']).divide(X['reg_opsum'])
    X['day_id'] = X['day_id'].map(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    X['dayofweek'] = X['day_id'].map(lambda x: x.dayofweek)
    return X

X_train, X_test = gen_features(train), gen_features(test)
# Fill NaN
X_train.fillna(value = 1, inplace = True)
X_test.fillna(value = 1, inplace = True)

In [5]:
def gen_categories(X_train, X_test):
    ohe = OneHotEncoder(sparse = False)
    temp = pd.concat([X_train[['lvl0_id']], X_test[['lvl0_id']]])['lvl0_id'].map(str)
    ohe.fit(temp.values.reshape(-1, 1))
    ohe_train = pd.DataFrame(ohe.transform(X_train['lvl0_id'].values.reshape(-1, 1)), index = X_train.index).join(X_train['txn_id'])
    ohe_test = pd.DataFrame(ohe.transform(X_test['lvl0_id'].values.reshape(-1, 1)), index = X_test.index).join(X_test['txn_id'])
    ohe_train = ohe_train.groupby('txn_id').sum()
    ohe_test = ohe_test.groupby('txn_id').sum()
    ohe_train.columns = ['lvl0_' + str(int(i)) for i in range(ohe_train.shape[1])]
    ohe_test.columns = ['lvl0_' + str(int(i)) for i in range(ohe_test.shape[1])]
    train = ohe_train
    test = ohe_test
    
    temp = pd.concat([X_train[['lvl1_id']], X_test[['lvl1_id']]])['lvl1_id'].map(str)
    ohe.fit(temp.values.reshape(-1, 1))
    ohe_train = pd.DataFrame(ohe.transform(X_train['lvl1_id'].values.reshape(-1, 1)), index = X_train.index).join(X_train['txn_id'])
    ohe_test = pd.DataFrame(ohe.transform(X_test['lvl1_id'].values.reshape(-1, 1)), index = X_test.index).join(X_test['txn_id'])
    ohe_train = ohe_train.groupby('txn_id').sum()
    ohe_test = ohe_test.groupby('txn_id').sum()
    ohe_train.columns = ['lvl1_' + str(int(i)) for i in range(ohe_train.shape[1])]
    ohe_test.columns = ['lvl1_' + str(int(i)) for i in range(ohe_test.shape[1])]
    train = train.join(ohe_train)
    test = test.join(ohe_test)
    
    temp = pd.concat([X_train[['lvl2_id']], X_test[['lvl2_id']]])['lvl2_id'].map(str)
    ohe.fit(temp.values.reshape(-1, 1))
    ohe_train = pd.DataFrame(ohe.transform(X_train['lvl2_id'].values.reshape(-1, 1)), index = X_train.index).join(X_train['txn_id'])
    ohe_test = pd.DataFrame(ohe.transform(X_test['lvl2_id'].values.reshape(-1, 1)), index = X_test.index).join(X_test['txn_id'])
    ohe_train = ohe_train.groupby('txn_id').sum()
    ohe_test = ohe_test.groupby('txn_id').sum()
    ohe_train.columns = ['lvl2_' + str(int(i)) for i in range(ohe_train.shape[1])]
    ohe_test.columns = ['lvl2_' + str(int(i)) for i in range(ohe_test.shape[1])]
    train = train.join(ohe_train)
    test = test.join(ohe_test)
    
    temp = pd.concat([X_train[['lvl3_id']], X_test[['lvl3_id']]])['lvl3_id'].map(str)
    ohe.fit(temp.values.reshape(-1, 1))
    ohe_train = pd.DataFrame(ohe.transform(X_train['lvl3_id'].values.reshape(-1, 1)), index = X_train.index).join(X_train['txn_id'])
    ohe_test = pd.DataFrame(ohe.transform(X_test['lvl3_id'].values.reshape(-1, 1)), index = X_test.index).join(X_test['txn_id'])
    ohe_train = ohe_train.groupby('txn_id').sum()
    ohe_test = ohe_test.groupby('txn_id').sum()
    ohe_train.columns = ['lvl3_' + str(int(i)) for i in range(ohe_train.shape[1])]
    ohe_test.columns = ['lvl3_' + str(int(i)) for i in range(ohe_test.shape[1])]
    train = train.join(ohe_train)
    test = test.join(ohe_test)
    
    # Удалим все дубликаты
    train = train.reset_index().drop_duplicates()
    train = train.set_index('txn_id')
    test = test.reset_index().drop_duplicates()
    test = test.set_index('txn_id')
    for column in train.columns:
        train[column] = train[column].map(lambda x: 0 if x == 0 else 1)
    for column in test.columns:
        test[column] = test[column].map(lambda x: 0 if x == 0 else 1)
    return train, test

def mk_new_data(X_train):
    X_train = X_train.copy()
    # Посчитаем количество товаров в чеке
    X_train['qty_int'] = X_train['sale_qnty'].map(lambda x: 1 if x != 0 else 0)
    # Просуммируем количество товаров, определим общую сумму чека
    train = X_train.groupby('txn_id')['qty', 'qty_int', 'reg_opsum', 'fact_opsum'].sum()
    
    # Добавим дату и день недели к данным
    if ('client_id' in X_train.columns):
        train = train.join(X_train.set_index('txn_id')[['day_id', 'dayofweek', 'mission_id', 'client_id']])
    else:
        train = train.join(X_train.set_index('txn_id')[['day_id', 'dayofweek', 'mission_id']])
    
    # Скидка по чеку
    train['prod_disc'] = (train['reg_opsum'] - train['fact_opsum']).divide(train['reg_opsum'])
    # Средняя и медианная скидки на товары в чеке
    disc_mean = X_train.groupby('txn_id')['prod_disc'].mean().rename('txn_disc_mean')
    disc_median = X_train.groupby('txn_id')['prod_disc'].median().rename('txn_disc_median')
    train = train.join(disc_mean)
    train = train.join(disc_median)
    # Определим среднюю стоимость товара в чеке
    train['avg'] = train['fact_opsum'].divide(train['qty'])
    disc_mean = X_train.groupby('txn_id')['fact_opsum'].mean().rename('txn_mean')
    disc_median = X_train.groupby('txn_id')['fact_opsum'].median().rename('txn_median')
    train = train.join(disc_mean)
    train = train.join(disc_median)
    
    # Отклонения от среднего чека
    # Цена
    txn_mean = train['fact_opsum'].mean()
    txn_median = train['fact_opsum'].median()
    train['txn_mean_diff'] = (train['fact_opsum'] / txn_mean).map(np.log1p)
    train['txn_median_diff'] = (train['fact_opsum'] / txn_median).map(np.log1p)
    # Количество товаров
    txn_qty_mean = train['qty'].mean()
    txn_qty_median = train['qty'].median()
    train['txn_mean_qty_diff'] = (train['qty'] / txn_qty_mean).map(np.log1p)
    train['txn_median_qty_diff'] = (train['qty'] / txn_qty_median).map(np.log1p)
    #Количество позиций в чеке
    txn_iqty_mean = train['qty_int'].mean()
    txn_iqty_median = train['qty_int'].median()
    train['txn_mean_iqty_diff'] = (train['qty_int'] / txn_iqty_mean).map(np.log1p)
    train['txn_median_iqty_diff'] = (train['qty_int'] / txn_iqty_median).map(np.log1p)
    
    # Отклонения от среднедневного и средненедельного чеков по цене
    txn_day_mean = train.groupby('day_id')['fact_opsum'].mean().rename('txn_mean_day_diff')
    train = train.join(txn_day_mean, on = 'day_id')
    txn_day_median = train.groupby('day_id')['fact_opsum'].median().rename('txn_median_day_diff')
    train = train.join(txn_day_median, on = 'day_id')
    train['txn_mean_day_diff'] = train['fact_opsum'].divide(train['txn_mean_day_diff'])
    train['txn_median_day_diff'] = train['fact_opsum'].divide(train['txn_median_day_diff'])
    
    week_mean = train.set_index('day_id').groupby('day_id')['fact_opsum'].mean().rolling(window = '7d').mean().rename('txn_mean_week_diff')
    week_median = train.set_index('day_id').groupby('day_id')['fact_opsum'].median().rolling(window = '7d').median().rename('txn_median_week_diff')
    train = train.join(week_mean, on = 'day_id')
    train = train.join(week_median, on = 'day_id')
    train['txn_mean_week_diff'] = train['fact_opsum'].divide(train['txn_mean_week_diff'])
    train['txn_median_week_diff'] = train['fact_opsum'].divide(train['txn_median_week_diff'])
    
    # Отклонения от среднедневного и средненедельного чеков по количеству
    qty_day_mean = train.groupby('day_id')['qty'].mean().rename('qty_mean_day_diff')
    train = train.join(qty_day_mean, on = 'day_id')
    qty_day_median = train.groupby('day_id')['qty'].median().rename('qty_median_day_diff')
    train = train.join(qty_day_median, on = 'day_id')
    train['qty_mean_day_diff'] = train['qty'].divide(train['qty_mean_day_diff'])
    train['qty_median_day_diff'] = train['qty'].divide(train['qty_median_day_diff'])
    
    week_mean = train.set_index('day_id').groupby('day_id')['qty'].mean().rolling(window = '7d').mean().rename('qty_mean_week_diff')
    week_median = train.set_index('day_id').groupby('day_id')['qty'].median().rolling(window = '7d').median().rename('qty_median_week_diff')
    train = train.join(week_mean, on = 'day_id')
    train = train.join(week_median, on = 'day_id')
    train['qty_mean_week_diff'] = train['qty'].divide(train['qty_mean_week_diff'])
    train['qty_median_week_diff'] = train['qty'].divide(train['qty_median_week_diff'])
    
    # Отклонения от среднедневного и средненедельного чеков по количеству позиций в чеке
    qty_day_mean = train.groupby('day_id')['qty'].mean().rename('iqty_mean_day_diff')
    train = train.join(qty_day_mean, on = 'day_id')
    qty_day_median = train.groupby('day_id')['qty'].median().rename('iqty_median_day_diff')
    train = train.join(qty_day_median, on = 'day_id')
    train['iqty_mean_day_diff'] = train['qty_int'].divide(train['iqty_mean_day_diff'])
    train['iqty_median_day_diff'] = train['qty_int'].divide(train['iqty_median_day_diff'])
    
    week_mean = train.set_index('day_id').groupby('day_id')['qty_int'].mean().rolling(window = '7d').mean().rename('iqty_mean_week_diff')
    week_median = train.set_index('day_id').groupby('day_id')['qty_int'].median().rolling(window = '7d').median().rename('iqty_median_week_diff')
    train = train.join(week_mean, on = 'day_id')
    train = train.join(week_median, on = 'day_id')
    train['iqty_mean_week_diff'] = train['qty_int'].divide(train['iqty_mean_week_diff'])
    train['iqty_median_week_diff'] = train['qty_int'].divide(train['iqty_median_week_diff'])
    
    # Средняя дневная и недельная скидка
    daily_disc = train.groupby('day_id')[['reg_opsum', 'fact_opsum']].sum()
    daily_disc = 1 - daily_disc['fact_opsum'].divide(daily_disc['reg_opsum']).rename('daily_disc')
    train = train.join(daily_disc, on = 'day_id')
    
    week_mean = train.set_index('day_id').groupby('day_id')['daily_disc'].mean().rolling(window = '7d').mean().rename('mean_week_disc')
    week_median = train.set_index('day_id').groupby('day_id')['daily_disc'].median().rolling(window = '7d').median().rename('median_week_disc')
    train = train.join(week_mean, on = 'day_id')
    train = train.join(week_median, on = 'day_id')
    
    # Отклонения скидок в чеке от средних значений
    train['diff_daily_disc'] = 1 - train['prod_disc'].divide(train['daily_disc'])
    train['diff_week_med_disc'] = 1 - train['prod_disc'].divide(train['median_week_disc'])
    train['diff_week_mean_disc'] = 1 - train['prod_disc'].divide(train['mean_week_disc'])
    # Попытаемся оценить, сколько товаров в чеке имеют скидки
    train['txn_disc_mean_diff'] = 1 - train['prod_disc'].divide(train['txn_disc_mean'])
    train['txn_disc_med_diff'] = 1 - train['prod_disc'].divide(train['txn_disc_median'])
    train['txn_disc_diff'] = 1 - train['txn_disc_mean'].divide(train['txn_disc_median'])
    train['txn_disc_mean_diff'] = train['txn_disc_mean_diff'].map(lambda x: 0 if x == -np.inf else 0 if (x == np.inf) else x)
    train['txn_disc_med_diff'] = train['txn_disc_med_diff'].map(lambda x: 0 if x == -np.inf else 0 if (x == np.inf) else x)
    train['txn_disc_diff'] = train['txn_disc_diff'].map(lambda x: 0 if x == -np.inf else 0 if (x == np.inf) else x)
    
    # Просто фантазия
    #train['disc_over_price'] = train['prod_disc'].divide(train['avg'])
    #train['disc_over_price1'] = train['prod_disc'].divide(train['txn_mean'])
    #train['disc_over_price2'] = train['prod_disc'].divide(train['txn_median'])
    #train['disc_over_txn'] = train['prod_disc'].divide(train['fact_opsum'])
    #train['disc_over_price'] = train['disc_over_price'].map(lambda x: 0 if x == -np.inf else 0 if (x == np.inf) else x)
    #train['disc_over_price1'] = train['disc_over_price1'].map(lambda x: 0 if x == -np.inf else 0 if (x == np.inf) else x)
    #train['disc_over_price2'] = train['disc_over_price2'].map(lambda x: 0 if x == -np.inf else 0 if (x == np.inf) else x)
    #train['disc_over_txn'] = train['disc_over_txn'].map(lambda x: 0 if x == -np.inf else 0 if (x == np.inf) else x)
    
    
    # Удалим дубликаты строк, если появились в процессе
    train = train.reset_index().drop_duplicates()
    train = train.set_index('txn_id')
    train.fillna(value = 0, inplace = True)
    train['day_id'] = train['day_id'].map(lambda x: x.timestamp())
    return train

train = mk_new_data(X_train)
test = mk_new_data(X_test)
train_cat, test_cat = gen_categories(X_train, X_test)
train = train.join(train_cat)
test = test.join(test_cat)
x_train = train.drop('client_id', axis = 1)
y_train = train['client_id']
print(train.shape)
train[:10]


(2043, 1366)


qty  qty_int  reg_opsum  fact_opsum        day_id  dayofweek  \
txn_id                                                                        
14873385393  0.260        1     83.174      83.174  1.517443e+09          3   
14874641148  1.841        4    448.716     406.550  1.517443e+09          3   
14875397469  0.056        1     25.900      22.000  1.517443e+09          3   
14875798817  1.766        3    170.400     145.300  1.517443e+09          3   
14875907886  3.371       10    851.773     744.841  1.517443e+09          3   
14877308791  1.730        3    105.500     105.500  1.517443e+09          3   
14878084937  1.340        3    237.369     205.469  1.517443e+09          3   
14878420047  1.057        5    192.500     171.500  1.517443e+09          3   
14878594133  0.562        2    155.164      79.563  1.517443e+09          3   
14878750721  3.496        8    394.900     367.200  1.517443e+09          3   

             mission_id  client_id  prod_disc  txn_disc_mean  txn_disc_median  \
txn_id                                                                          
14873385393          -1   96870188   0.000000       0.000000         0.000000   
14874641148          -1  162687105   0.093970       0.070585         0.066146   
14875397469          -1  116871314   0.150579       0.150579         0.150579   
14875798817          -1  132059478   0.147300       0.125437         0.000000   
14875907886          -1  153161102   0.125540       0.177944         0.043178   
14877308791          -1  153175485   0.000000       0.000000         0.000000   
14878084937          -1  117318308   0.134390       0.165268         0.220441   
14878420047          -1  159953160   0.109091       0.076699         0.000000   
14878594133          -1  120532784   0.487233       0.438620         0.438620   
14878750721          -1  153157800   0.070144       0.069944         0.000000   

                    avg    txn_mean  txn_median  txn_mean_diff  \
txn_id                                                           
14873385393  319.900000   83.174000     83.1740       0.224183   
14874641148  220.831070  101.637500    104.2450       0.801257   
14875397469  392.857143   22.000000     22.0000       0.064354   
14875798817   82.276331   48.433333     41.6000       0.363953   
14875907886  220.955503   74.484100     51.3500       1.178792   
14877308791   60.982659   35.166667     29.9000       0.276688   
14878084937  153.335075   68.489667     55.0000       0.482919   
14878420047  162.251656   34.300000     36.5000       0.417503   
14878594133  141.571174   39.781500     39.7815       0.215426   
14878750721  105.034325   45.900000     39.9000       0.746427   

             txn_median_diff  txn_mean_qty_diff  txn_median_qty_diff  \
txn_id                                                                 
14873385393         0.281380           0.096572             0.121304   
14874641148         0.951029           0.541109             0.648774   
14875397469         0.082458           0.021603             0.027399   
14875798817         0.449597           0.523938             0.629138   
14875907886         1.363552           0.839213             0.982873   
14877308791         0.345137           0.515590             0.619573   
14878084937         0.589317           0.420381             0.509634   
14878420047         0.512849           0.345138             0.421539   
14878594133         0.270675           0.198159             0.245899   
14878750721         0.889796           0.860054             1.005812   

             txn_mean_iqty_diff  txn_median_iqty_diff  txn_mean_day_diff  \
txn_id                                                                     
14873385393            0.153148              0.182322           0.260132   
14874641148            0.508016              0.587787           1.271509   
14875397469            0.153148              0.182322           0.068806   
14875798817            0.403124              0.470004  

def clusterize(X_train, X_test, N = 50, label = 'clust50_'):
    if X_train.shape[1] != X_test.shape[1]:
        print('size  mismatch')
        return
    kmeans = KMeans(n_clusters = N, init = 'k-means++', random_state = 42)
    ohe = OneHotEncoder(sparse = False)
    X_train = X_train.copy()
    X_test = X_test.copy()
    train = kmeans.fit_predict(X_train)
    test = kmeans.predict(X_test)
    train = pd.DataFrame(ohe.fit_transform(train.reshape(-1, 1)), index = X_train.index)
    test = pd.DataFrame(ohe.transform(test.reshape(-1, 1)), index = X_test.index)
    train.columns = [label + str(int(i)) for i in range(train.shape[1])]
    test.columns = [label + str(int(i)) for i in range(test.shape[1])]
    return train, test

x_train_t, x_test_t = clusterize(x_train, test, 3, 'clust3_')
x_train_c = x_train.join(x_train_t)
x_test_c = test.join(x_test_t)
x_train_t, x_test_t = clusterize(x_train, test, 5, 'clust5_')
x_train_c = x_train_c.join(x_train_t)
x_test_c = x_test_c.join(x_test_t)
x_train_t, x_test_t = clusterize(x_train, test, 15, 'clust15_')
x_train_c = x_train_c.join(x_train_t)
x_test_c = x_test_c.join(x_test_t)
x_train_t, x_test_t = clusterize(x_train, test, 30, 'clust30_')
x_train_c = x_train_c.join(x_train_t)
x_test_c = x_test_c.join(x_test_t)

#train_clust, test_clust = clusterize(x_train, test)

train.groupby(pd.Grouper(key='day_id', freq='W'))['fact_opsum'].mean().rename('fact_opsum_avg_week')

In [6]:
print('Splitting')
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(x_train, y_train, test_size=0.1, shuffle = True)
def print_nan(X):
    counts = X.count()
    length = X.shape[0]
    for i in counts[counts < length].index:
        print(i, ':', '%.3f' % (counts[i] / length))
    return
#print_nan(X_train_split)

gbc = GradientBoostingClassifier(n_estimators = 5000, learning_rate = 0.01, random_state = 42)
print('Fitting')
gbc.fit(X_train_split, y_train_split)
print('Predicting')
y_predicted = gbc.predict(X_test_split)
print('accuracy:', accuracy_score(y_test_split, y_predicted))
print('recall:', recall_score(y_test_split, y_predicted, average = 'micro'))
print('precision:', accuracy_score(y_test_split, y_predicted))
print('f1:', accuracy_score(y_test_split, y_predicted))
print('zero_one_loss:', zero_one_loss(y_test_split, y_predicted))
# print(classification_report(y_test_split, y_predicted))

Splitting
Fitting
Predicting
accuracy: 0.43414634146341463
recall: 0.43414634146341463
precision: 0.43414634146341463
f1: 0.43414634146341463
zero_one_loss: 0.5658536585365854


# Построим попарные графики и посмотрим, насколько вообще визуально разделимы группы чеков. Не очень.
import pylab
colors = [int(i % y_train['client_id'].unique().shape[0]) for i in y_train.reset_index(drop = True).index]
for i in train.columns:
    for j in train.columns:
        pylab.scatter(train_clust[i], train_clust[j], c = colors)
        pylab.xlabel(i)
        pylab.ylabel(i)
        pylab.show()

In [9]:
gbc = GradientBoostingClassifier(n_estimators = 5000, learning_rate = 0.01, random_state = 42)
gbc.fit(x_train, y_train)
f_imp = pd.Series(gbc.feature_importances_, index = x_train.columns)
f_imp = f_imp.sort_values(ascending = False)
for column in f_imp.index:
    print ('{:10s}\t {:2.8f}'.format(column, 100 * f_imp[column]))


avg       	 3.05379214
txn_median	 2.01666570
day_id    	 1.58077540
daily_disc	 1.48469957
iqty_mean_day_diff	 1.34778797
iqty_mean_week_diff	 1.33571495
iqty_median_day_diff	 1.30428195
txn_mean  	 1.28253994
txn_median_day_diff	 1.14987799
qty_median_day_diff	 1.01351862
txn_median_week_diff	 0.92642289
reg_opsum 	 0.91845574
mean_week_disc	 0.89614606
qty_mean_week_diff	 0.88524917
txn_disc_mean_diff	 0.87807400
qty_median_week_diff	 0.86427009
txn_mean_day_diff	 0.82775242
txn_mean_week_diff	 0.79765045
qty_mean_day_diff	 0.78949716
median_week_disc	 0.74609243
txn_disc_med_diff	 0.73449580
diff_daily_disc	 0.72728805
txn_disc_mean	 0.67918082
txn_disc_median	 0.64662903
prod_disc 	 0.62530154
txn_median_qty_diff	 0.60584258
txn_mean_qty_diff	 0.59517659
diff_week_mean_disc	 0.59073050
fact_opsum	 0.58966415
qty       	 0.58838937
txn_mean_diff	 0.58691853
txn_median_diff	 0.57931449
diff_week_med_disc	 0.54353153
txn_disc_diff	 0.54083874
iqty_median_week_diff	 0.48247296
mission

In [11]:
y_predict = pd.DataFrame(gbc.predict(test), index = test.index, columns = ['client_id'])

In [12]:
test = pd.read_csv('clients_test_cut.csv')
test = test.join(y_predict, on = 'txn_id')
test['client_id'].to_csv('prediction-5000-001.csv')